In [1]:
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect('carmen.sqlite')

In [3]:
pd.read_sql("select name from sqlite_master;", con=conn)

,name
0,city
1,country
2,countrylanguage


In [4]:
pd.read_sql("select * from city;", con=conn).head(2)

,id,name,countrycode,district,population
0,1,Kabul,AFG,Kabol,1780000
1,2,Qandahar,AFG,Qandahar,237500


In [5]:
pd.read_sql("select * from country;", con=conn).head(2)

,code,name,continent,region,surfacearea,indepyear,population,lifeexpectancy,gnp,gnpold,localname,governmentform,headofstate,capital,code2
0,AFG,Afghanistan,Asia,Southern and Central Asia,652090.0,1919.0,22720000,45.9,5976.0,NaN,Afganistan/Afqanestan,Islamic Emirate,Mohammad Omar,1.0,AF
1,NLD,Netherlands,Europe,Western Europe,41526.0,1581.0,15864000,78.3,371362.0,360478.0,Nederland,Constitutional Monarchy,Beatrix,5.0,NL


In [6]:
pd.read_sql("select * from countrylanguage;", con=conn).head(2)

,countrycode,language,isofficial,percentage
0,AFG,Pashto,1,52.4
1,NLD,Dutch,1,95.6


# Clue #1

We recently got word that someone fitting Carmen Sandiego's description has been
traveling through Southern Europe. She's most likely traveling someplace where she won't be noticed, so find the least populated country in Southern Europe, and we'll start looking for her there.

In [7]:
query = """
select * 
from country 
where region like 'Southern Europe'
order by population asc
limit 1;
"""

pd.read_sql(query, con=conn)

,code,name,continent,region,surfacearea,indepyear,population,lifeexpectancy,gnp,gnpold,localname,governmentform,headofstate,capital,code2
0,VAT,Holy See (Vatican City State),Europe,Southern Europe,0.4,1929.0,1000,None,9.0,None,Santa Sede/Cittï¿½ del Vaticano,Independent Church State,Johannes Paavali II,3538.0,VA


# Clue #2

 Now that we're here, we have insight that Carmen was seen attending language classes in
this country's officially recognized language. Check our databases and find out what language is spoken in this country, so we can call in a translator to work with you.

In [8]:
query = """
select * from countrylanguage where countrycode = 'VAT';
"""

pd.read_sql(query, con=conn)

,countrycode,language,isofficial,percentage
0,VAT,Italian,1,0.0


# Clue #3

We have new news on the classes Carmen attended – our gumshoes tell us she's moved on to a different country, a country where people speak only the language she was learning. Find out which nearby country speaks nothing but that language.

In [9]:
query = """
select * from countrylanguage where language = 'Italian' and percentage=100;
"""

pd.read_sql(query, con=conn)

,countrycode,language,isofficial,percentage
0,SMR,Italian,1,100.0


# Clue #4

We're booking the first flight out – maybe we've actually got a chance to catch her this time. There are only two cities she could be flying to in the country. One is named the same as the country – that would be too obvious. We're following our gut on this one; find out what other city in that country she might be flying to.

In [10]:
query = """
select * from city where countrycode = 'SMR';
"""

pd.read_sql(query, con=conn)

,id,name,countrycode,district,population
0,3170,Serravalle,SMR,Serravalle/Dogano,4802
1,3171,San Marino,SMR,San Marino,2294


# Clue #5

Oh no, she pulled a switch – there are two cities with very similar names, but in totally different parts of the globe! She's headed to South America as we speak; go find a city whose name is like the one we were headed to, but doesn't end the same. Find out the city, and do another search for what country it's in. Hurry!

In [11]:
query = """
select 
    co.code, 
    co.name as country, 
    co.continent, 
    co.region,
    ci.name as city
from city as ci
left join country as co on co.code = ci.countrycode
where ci.name like 'Ser%' and region = 'South America';
"""

pd.read_sql(query, con=conn)

,code,country,continent,region,city
0,BRA,Brazil,South America,South America,Serra
1,BRA,Brazil,South America,South America,Sertï¿½ozinho


# Clue #6

We're close! Our South American agent says she just got a taxi at the airport, and is headed towards the capital! Look up the country's capital, and get there pronto! Send us the name of where you're headed and we'll follow right behind you!

In [12]:
query = """
select 
    co.code, 
    co.name as country,
    ci.name as capital
from city as ci
left join country as co on co.code = ci.countrycode and co.capital=ci.id
where co.name='Brazil';
"""

pd.read_sql(query, con=conn)

,code,country,capital
0,BRA,Brazil,Brasï¿½lia


# Clue #7

She knows we're on to her – her taxi dropped her off at the international airport, and she beat us to the boarding gates. We have one chance to catch her, we just have to know where she's heading and beat her to the landing dock.

Lucky for us, she's getting cocky. She left us a note, and I'm sure she thinks she's very clever, but if we can crack it, we can finally put her where she belongs – behind bars.
```
Our playdate of late has been unusually fun –
As an agent, I'll say, you've been a joy to outrun.
And while the food here is great, and the people – so nice!
I need a little more sunshine with my slice of life.
So I'm off to add one to the population I find
In a city of ninety-one thousand and now, eighty five.
```
We're counting on you, gumshoe. Find out where she's headed, send us the info, and we'll be sure to meet her at the gates with bells on.

In [13]:
query = """
select * from city WHERE population = 91084;
"""

pd.read_sql(query, con=conn)

,id,name,countrycode,district,population
0,4060,Santa Monica,USA,California,91084


*She's in Santa Monica!*